Model 4 부분이 제가 보낸 모델입니다

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조

/content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조


In [8]:
raw_data = pd.read_csv('common_data.csv')
raw_data.head(3)

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타혐오,악플/욕설,clean,분쟁유발,교정문장,띄어쓰기수정문장
0,문재앙은 김정은의 운전사도 안되는 똥휴지다 개돼지들이 뽑은 재앙민국이 망해간다,0,0,0,0,0,0,0,0,1,0,1.0,물 재앙은 김정은의 운전사도 안되는 똥 휴지다 개돼지들이 뽑은 재앙 민국이 망해간다,문재앙은 김정은의 운전사도 안 되는 똥 휴지 다 개 돼지들이 뽑은 재앙민국이 망해간다
1,근데 사람들이 우스갯소리로 일 재앙이라고 했는데 이제보니 장난아니고 일 재앙...,0,0,0,0,0,0,0,0,0,0,1.0,근데 사람들이 우스갯소리로 일 재앙이라고 했는데 이제 보니 장난 아니고 일 ...,근데 사람들이 우스갯소리로 일 재앙이라고 했는데 이 제보니 장난 아니고 일 재앙은 ...
2,공무뭔들도 정권 바뀐다는걸 아는구나 이놈들은 원래 연줄이 밥줄이라 눈치는 백단이지 ...,0,0,0,0,0,0,0,0,1,0,1.0,공무 뭔들도 정권 바뀐다는 걸 아는구나 이놈들은 원래 연줄이 밥줄이라 눈치는 백단이...,공무뭔들도 정권 바뀐다는 걸 아는구나 이놈들은 원래 연줄이 밥줄이라 눈치는 백단이지...


In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

In [9]:
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
%cd /content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조

mecab = Mecab()
f = open('stopwords.txt', 'r')
stopwords = []
while True:
  temp = f.readline().strip()
  if not temp:
    break
  stopwords.append(temp)

/content/drive/MyDrive/멀티캠퍼스 2차 프로젝트 5조


In [131]:
# 데이터셋 나누기는 해야함
raw_data = raw_data.sample(len(raw_data))
X_train = raw_data.iloc[:37925, 0]
y_train = raw_data.iloc[:37925, 1:-2]
X_test = raw_data.iloc[37925:, 0]
y_test = raw_data.iloc[37925:, 1:-2]

In [132]:
train_data = []
for sentence in tqdm(X_train):
  morphs = mecab.morphs(sentence)
  tmp_X = [word for word in morphs if word not in stopwords]
  train_data.append(tmp_X)

  0%|          | 0/37925 [00:00<?, ?it/s]

In [133]:
test_data = []
for sentence in tqdm(X_test):
  morphs = mecab.morphs(sentence)
  tmp_X = [word for word in morphs if word not in stopwords]
  test_data.append(tmp_X)

  0%|          | 0/9484 [00:00<?, ?it/s]

In [14]:
import pickle

t = pd.read_pickle('tokenizer.pkl')

In [16]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)
X_train = t.texts_to_sequences(train_data)
X_test = t.texts_to_sequences(test_data)

NameError: ignored

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 70 # 그냥 문장은 70이 제일 높음, 스페이스 교정 문장도 마찬가진데?
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

X_train.shape, X_test.shape

NameError: ignored

In [137]:
vocab_size = len(t.word_index) + 1
vocab_size

40716

In [138]:
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, GlobalMaxPooling1D, Dropout, Concatenate, MaxPooling1D, GRU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### Model 1

*   Embedding
*   Conv1D
*   Conv1D



In [ ]:
model1 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    Conv1D(64, 5, padding='valid', activation='relu', strides=1),
    Dropout(0.5),
    Conv1D(32, 5, padding='valid', activation='relu', strides=1),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dense(11, activation='softmax')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 128)           5211648   
                                                                 
 conv1d (Conv1D)             (None, 56, 64)            41024     
                                                                 
 dropout (Dropout)           (None, 56, 64)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 52, 32)            10272     
                                                                 
 dropout_1 (Dropout)         (None, 52, 32)            0         
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                        

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1_path = 'best-cnn.h5py'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [ ]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, 
    batch_size=128,
    callbacks=[mc1, es1], 
    verbose=0
)


Epoch 1: val_loss improved from inf to 390.75558, saving model to best-cnn.h5py
INFO:tensorflow:Assets written to: best-cnn.h5py/assets

Epoch 2: val_loss did not improve from 390.75558

Epoch 3: val_loss did not improve from 390.75558

Epoch 4: val_loss did not improve from 390.75558

Epoch 5: val_loss did not improve from 390.75558

Epoch 6: val_loss did not improve from 390.75558


In [ ]:
best_model1 = load_model(model1_path)
best_model1.evaluate(X_test, y_test)

297/297 [==============================] - 1s 3ms/step - loss: 393.6883 - accuracy: 0.1145


[393.6882629394531, 0.11453279852867126]

### Model 2

*   Embedding
*   Conv1D
*   Conv1D
*   LSTM



In [ ]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
model2 = Sequential([ 
    Embedding(vocab_size, 200, input_length=max_len),
    Bidirectional(LSTM(200, return_sequences=True)),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(11, activation='softmax')
])
model2.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 60, 200)           8143200   
                                                                 
 bidirectional_2 (Bidirectio  (None, 60, 400)          641600    
 nal)                                                            
                                                                 
 dropout_22 (Dropout)        (None, 60, 400)           0         
                                                                 
 global_max_pooling1d_9 (Glo  (None, 400)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_20 (Dense)            (None, 11)                4411      
                                                                 
Total params: 8,789,211
Trainable params: 8,789,211
N

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2_path = 'best-biLSTM.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [ ]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=128, callbacks=[mc2,es2]
)

Epoch 1/30
238/238 [==============================] - ETA: 0s - loss: 2.9744 - accuracy: 0.1185
Epoch 1: val_loss improved from inf to 2.92296, saving model to best-biLSTM.h5
238/238 [==============================] - 10s 31ms/step - loss: 2.9744 - accuracy: 0.1185 - val_loss: 2.9230 - val_accuracy: 0.1297
Epoch 2/30
237/238 [============================>.] - ETA: 0s - loss: 2.7365 - accuracy: 0.1710
Epoch 2: val_loss improved from 2.92296 to 2.75075, saving model to best-biLSTM.h5
238/238 [==============================] - 7s 27ms/step - loss: 2.7364 - accuracy: 0.1710 - val_loss: 2.7507 - val_accuracy: 0.1950
Epoch 3/30
236/238 [============================>.] - ETA: 0s - loss: 2.4995 - accuracy: 0.2662
Epoch 3: val_loss improved from 2.75075 to 2.57449, saving model to best-biLSTM.h5
238/238 [==============================] - 7s 28ms/step - loss: 2.4988 - accuracy: 0.2662 - val_loss: 2.5745 - val_accuracy: 0.3127
Epoch 4/30
238/238 [==============================] - ETA: 0s - loss: 

In [ ]:
best_model2 = load_model(model2_path)
best_model2.evaluate(X_test, y_test)

297/297 [==============================] - 3s 7ms/step - loss: 2.5703 - accuracy: 0.3148


[2.5703063011169434, 0.3148069977760315]

### Model 3

*   Embedding
*   LSTM



In [ ]:
model3 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    LSTM(128),
    Dense(11, activation='softmax')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 60, 128)           5211648   
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_3 (Dense)             (None, 11)                1419      
                                                                 
Total params: 5,344,651
Trainable params: 5,344,651
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.compile('adam', 'categorical_crossentropy', ['accuracy'])
model3_path = 'best-lstm.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [ ]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc3, es3]
)

Epoch 1/100
470/475 [============================>.] - ETA: 0s - loss: 2.9667 - accuracy: 0.1167
Epoch 1: val_loss improved from inf to 2.97062, saving model to best-lstm.h5
475/475 [==============================] - 6s 12ms/step - loss: 2.9682 - accuracy: 0.1164 - val_loss: 2.9706 - val_accuracy: 0.1175
Epoch 2/100
473/475 [============================>.] - ETA: 0s - loss: 2.9684 - accuracy: 0.1164
Epoch 2: val_loss did not improve from 2.97062
475/475 [==============================] - 5s 11ms/step - loss: 2.9685 - accuracy: 0.1164 - val_loss: 2.9743 - val_accuracy: 0.1175
Epoch 3/100
475/475 [==============================] - ETA: 0s - loss: 2.9675 - accuracy: 0.1158
Epoch 3: val_loss did not improve from 2.97062
475/475 [==============================] - 9s 18ms/step - loss: 2.9675 - accuracy: 0.1158 - val_loss: 2.9822 - val_accuracy: 0.1175
Epoch 4/100
473/475 [============================>.] - ETA: 0s - loss: 2.9689 - accuracy: 0.1164
Epoch 4: val_loss improved from 2.97062 to 2.

In [ ]:
best_model3 = load_model(model3_path)
best_model3.evaluate(X_test, y_test)

297/297 [==============================] - 2s 5ms/step - loss: 2.9528 - accuracy: 0.1145


[2.952800750732422, 0.11453279852867126]

### Model 4

*   Embedding
*   Conv1D
*   LSTM

*이게 제가 보낸 모델입니다*

In [139]:
model4 = Sequential([
    Embedding(vocab_size, 128, input_length=max_len),
    Dropout(0.2),
    Conv1D(256, 3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=4),
    LSTM(128),
    Dense(11, activation='softmax')
])
model4.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 70, 128)           5211648   
                                                                 
 dropout_14 (Dropout)        (None, 70, 128)           0         
                                                                 
 conv1d_14 (Conv1D)          (None, 68, 256)           98560     
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 17, 256)          0         
 g1D)                                                            
                                                                 
 lstm_14 (LSTM)              (None, 128)               197120    
                                                                 
 dense_14 (Dense)            (None, 11)                1419      
                                                     

In [140]:
model4.compile('adam', 'categorical_crossentropy', ['accuracy'])
model4_path = 'best-conv1d_lstm.h5'
mc4 = ModelCheckpoint(model4_path, verbose=1, save_best_only=True)
es4 = EarlyStopping(patience=3)

In [141]:
hist4 = model4.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc4, es4]
)

Epoch 1/100
472/475 [============================>.] - ETA: 0s - loss: 2.4141 - accuracy: 0.3172
Epoch 1: val_loss improved from inf to 1.82362, saving model to best-conv1d_lstm.h5
475/475 [==============================] - 7s 11ms/step - loss: 2.4104 - accuracy: 0.3191 - val_loss: 1.8236 - val_accuracy: 0.6144
Epoch 2/100
470/475 [============================>.] - ETA: 0s - loss: 1.6233 - accuracy: 0.6737
Epoch 2: val_loss improved from 1.82362 to 1.74690, saving model to best-conv1d_lstm.h5
475/475 [==============================] - 5s 11ms/step - loss: 1.6240 - accuracy: 0.6736 - val_loss: 1.7469 - val_accuracy: 0.6519
Epoch 3/100
474/475 [============================>.] - ETA: 0s - loss: 1.4881 - accuracy: 0.7283
Epoch 3: val_loss did not improve from 1.74690
475/475 [==============================] - 5s 11ms/step - loss: 1.4888 - accuracy: 0.7283 - val_loss: 1.8549 - val_accuracy: 0.6705
Epoch 4/100
475/475 [==============================] - ETA: 0s - loss: 1.4927 - accuracy: 0.75

In [142]:
best_model4 = load_model(model4_path)
best_model4.evaluate(X_test, y_test)

297/297 [==============================] - 2s 4ms/step - loss: 1.6748 - accuracy: 0.6606


[1.67482590675354, 0.6605862379074097]

### Model 5

*   Embedding
*   GRU

In [ ]:
model5 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    GRU(128),
    Dense(11, activation='softmax')
])
model5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 60, 128)           5211648   
                                                                 
 gru (GRU)                   (None, 128)               99072     
                                                                 
 dense_5 (Dense)             (None, 11)                1419      
                                                                 
Total params: 5,312,139
Trainable params: 5,312,139
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model5.compile('adam', 'categorical_crossentropy', ['accuracy'])
model5_path = 'best-gru.h5'
mc5 = ModelCheckpoint(model5_path, verbose=1, save_best_only=True)
es5 = EarlyStopping(patience=5)

In [ ]:
hist5 = model5.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc5, es5]
)

Epoch 1/100
474/475 [============================>.] - ETA: 0s - loss: 2.9829 - accuracy: 0.1165
Epoch 1: val_loss improved from inf to 2.97242, saving model to best-gru.h5
475/475 [==============================] - 11s 18ms/step - loss: 2.9828 - accuracy: 0.1165 - val_loss: 2.9724 - val_accuracy: 0.1175
Epoch 2/100
470/475 [============================>.] - ETA: 0s - loss: 2.9691 - accuracy: 0.1164
Epoch 2: val_loss did not improve from 2.97242
475/475 [==============================] - 5s 11ms/step - loss: 2.9693 - accuracy: 0.1164 - val_loss: 2.9749 - val_accuracy: 0.1175
Epoch 3/100
473/475 [============================>.] - ETA: 0s - loss: 2.9677 - accuracy: 0.1158
Epoch 3: val_loss did not improve from 2.97242
475/475 [==============================] - 6s 12ms/step - loss: 2.9678 - accuracy: 0.1158 - val_loss: 2.9823 - val_accuracy: 0.1175
Epoch 4/100
471/475 [============================>.] - ETA: 0s - loss: 2.9692 - accuracy: 0.1164
Epoch 4: val_loss improved from 2.97242 to 2.

In [ ]:
best_model5 = load_model(model5_path)
best_model5.evaluate(X_test, y_test)

297/297 [==============================] - 2s 4ms/step - loss: 2.9528 - accuracy: 0.1145


[2.952824354171753, 0.11453279852867126]

### Model 6

*   Embedding
*   Conv1D
*   GRU

In [ ]:
model6 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    Dropout(0.2),
    Conv1D(256, 3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=4),
    GRU(128),
    Dense(11, activation='softmax')
])
model6.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 60, 128)           5211648   
                                                                 
 dropout_5 (Dropout)         (None, 60, 128)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 58, 256)           98560     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 14, 256)          0         
 1D)                                                             
                                                                 
 gru_1 (GRU)                 (None, 128)               148224    
                                                                 
 dense_6 (Dense)             (None, 11)                1419      
                                                      

In [ ]:
model6.compile('adam', 'categorical_crossentropy', ['accuracy'])
model6_path = 'best-conv1d_gru.h5'
mc6 = ModelCheckpoint(model6_path, verbose=1, save_best_only=True)
es6 = EarlyStopping(patience=5)

In [ ]:
hist6 = model6.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc6, es6]
)

Epoch 1/100
471/475 [============================>.] - ETA: 0s - loss: 2.3482 - accuracy: 0.3759
Epoch 1: val_loss improved from inf to 1.86814, saving model to best-conv1d_gru.h5
475/475 [==============================] - 7s 11ms/step - loss: 2.3461 - accuracy: 0.3773 - val_loss: 1.8681 - val_accuracy: 0.6030
Epoch 2/100
472/475 [============================>.] - ETA: 0s - loss: 1.7007 - accuracy: 0.6677
Epoch 2: val_loss did not improve from 1.86814
475/475 [==============================] - 5s 10ms/step - loss: 1.7012 - accuracy: 0.6678 - val_loss: 1.8712 - val_accuracy: 0.6411
Epoch 3/100
471/475 [============================>.] - ETA: 0s - loss: 1.6382 - accuracy: 0.7172
Epoch 3: val_loss did not improve from 1.86814
475/475 [==============================] - 4s 9ms/step - loss: 1.6381 - accuracy: 0.7170 - val_loss: 2.1357 - val_accuracy: 0.6450
Epoch 4/100
473/475 [============================>.] - ETA: 0s - loss: 1.6493 - accuracy: 0.7417
Epoch 4: val_loss did not improve from 1

In [ ]:
best_model6 = load_model(model6_path)
best_model6.evaluate(X_test, y_test)

297/297 [==============================] - 1s 4ms/step - loss: 1.8488 - accuracy: 0.6069


[1.848807454109192, 0.606939435005188]